## Importing libraries and reading the `.txt` file

In [12]:
from dataclasses import dataclass
from typing import List, Tuple
import cplex

from recordclass import recordclass
import sys



input_filename = "test_A.txt"

# Global parameters
add_soft_variables = True  #whether to include soft constraints or not
soft_conflict_penalty = 1  #penalty for assigning conflicting workers
soft_repetition_penalty = -1  #penalty for assigning repeated orders
M = 1e10
TOLERANCE = 1e-6



## Setting constants

In [13]:
# Order record
Order = recordclass('Order', 'oid benefit n_workers')

class CrewAssignmentInstance:
    def __init__(self):
        self.n_workers = 0
        self.n_orders = 0
        self.orders = []
        self.worker_conflicts = []
        self.order_chains = []
        self.order_conflicts = []
        self.order_repeats = []
        
    def read_data(self, filename):

        #open file
        f = open(filename)

        #number of workers
        self.n_workers = int(f.readline())
        
        #number of orders
        self.n_orders = int(f.readline())
        
        #orders
        self.orders = []
        for i in range(self.n_orders):
            line = f.readline().rstrip().split(' ')
            self.orders.append(Order(line[0], line[1], line[2]))
        
        #number of worker conflicts
        n_worker_conflicts = int(f.readline())
        
        #worker conflicts
        self.worker_conflicts = []
        for i in range(n_worker_conflicts):
            line = f.readline().split(' ')
            self.worker_conflicts.append(list(map(int, line)))
            
        #number of chained orders (B most be done after A)
        n_order_chains = int(f.readline())
        
        #chained orders
        self.order_chains = []
        for i in range(n_order_chains):
            line = f.readline().split(' ')
            self.order_chains.append(list(map(int, line)))
            
        #number of conflicting orders
        n_order_conflicts = int(f.readline())
        
        # conflicting orders
        self.order_conflicts = []
        for i in range(n_order_conflicts):
            line = f.readline().split(' ')
            self.order_conflicts.append(list(map(int, line)))
        
        #number of repeated orders
        n_order_repeats = int(f.readline())
        
        #repeated orders
        self.order_repeats = []
        for i in range(n_order_repeats):
            line = f.readline().split(' ')
            self.order_repeats.append(list(map(int, line)))
            
        # close file
        f.close()



#### Reading the file and defining model variables

In [14]:

def load_instance():
    instance = CrewAssignmentInstance()
    instance.read_data(input_filename)
    return instance


def add_variables(prob, instance):
    #define and add variables:
    #method 'add' from 'variables', with parameters:
    #obj: objective function coefficients
    #lb: lower bounds
    #ub: upper bounds
    #types: variable types
    #names: variable names (as they appear in .lp file)
    
    n_workers = instance.n_workers
    n_orders = instance.n_orders
    
    obj_coeffs = []
    names = []
    types = []
    lower = []
    upper = []
    
    #w_xj: number of hours worked in salary range x
    for j in range(n_workers):
        names.append("W 1 " + str(j))
        obj_coeffs.append(-1000)
        types.append("I")
        lower.append(0)
        upper.append(5)
    for j in range(n_workers):
        names.append("W 2 " + str(j))
        obj_coeffs.append(-1200)
        types.append("I")
        lower.append(0)
        upper.append(5)
    for j in range(n_workers):
        names.append("W 3 " + str(j))
        obj_coeffs.append(-1400)
        types.append("I")
        lower.append(0)
        upper.append(5)
    for j in range(n_workers):
        names.append("W 4 " + str(j))
        obj_coeffs.append(-1500)
        types.append("I")
        lower.append(0)
        upper.append(15)
        
    #o_ik = 1 if order i is done in shift k, 0 otherwise
    for i in range(n_orders):
        for k in range(1, 31):
            names.append("O " + str(int(instance.orders[i].oid)) + " " + str(k))
            obj_coeffs.append(0)
            types.append("B")
            lower.append(0)
            upper.append(1)
            
    #a_ij = 1 if order i is done by worker j, 0 otherwise
    for i in range(n_orders):
        for j in range(n_workers):
            names.append("A " + instance.orders[i].oid + " " + str(j))
            obj_coeffs.append(0)
            types.append("B")
            lower.append(0)
            upper.append(1)
            
    #y_xj = 1 if worker j worked in salary range x, 0 otherwise
    for j in range(n_workers):
        names.append("Y 1 " + str(j))
        obj_coeffs.append(0)
        types.append("B")
        lower.append(0)
        upper.append(1)
    for j in range(n_workers):
        names.append("Y 2 " + str(j))
        obj_coeffs.append(0)
        types.append("B")
        lower.append(0)
        upper.append(1)
    for j in range(n_workers):
        names.append("Y 3 " + str(j))
        obj_coeffs.append(0)
        types.append("B")
        lower.append(0)
        upper.append(1)
    for j in range(n_workers):
        names.append("Y 4 " + str(j))
        obj_coeffs.append(0)
        types.append("B")
        lower.append(0)
        upper.append(1)
        
    #x_kj = 1 if worker j worked in shift k, 0 otherwise
    for k in range(1, 31):
        for j in range(n_workers):
            names.append("X " + str(k) + " " + str(j))
            obj_coeffs.append(0)
            types.append("B")
            lower.append(0)
            upper.append(1)
            
    #v_dj = 1 if worker j worked on day d, 0 otherwise
    for d in range(1, 7):
        for j in range(n_workers):
            names.append("V " + str(d) + " " + str(j))
            obj_coeffs.append(0)
            types.append("B")
            lower.append(0)
            upper.append(1)
            
    #delta_ikj = 1 if worker j performed task i in shift k, 0 otherwise
    for i in range(n_orders):
        for k in range(1, 31):
            for j in range(n_workers):
                names.append("delta " + instance.orders[i].oid + " " + str(k) + " " + str(j))
                obj_coeffs.append(0)
                types.append("B")
                lower.append(0)
                upper.append(1)
                
    #number of hours worked by the most loaded worker
    names.append("more_work")
    obj_coeffs.append(0)
    types.append("I")
    lower.append(0)
    upper.append(n_orders)
    
    #number of hours worked by the least loaded worker
    names.append("less_work")
    obj_coeffs.append(0)
    types.append("I")
    lower.append(0)
    upper.append(n_orders)
    
    #h_i = 1 if order i was completed, 0 otherwise
    for i in range(n_orders):
        names.append("H " + instance.orders[i].oid)
        obj_coeffs.append(int(instance.orders[i].benefit))
        types.append("B")
        lower.append(0)
        upper.append(1)
    
    if add_soft_variables:
        #lambda_ijj' = 1 if worker j does task i with j' (they don't get along)
        for i in range(n_orders):
            for j, jprime in instance.worker_conflicts:
                names.append("lambda " + instance.orders[i].oid + " " + str(j) + " " + str(jprime))
                obj_coeffs.append(-coef_conflictos_trabajadores)
                types.append("B")
                lower.append(0)
                upper.append(1)
        
        #theta_ii'j = 1 if worker j does both tasks i and i' (they're repetitive)
        for j in range(n_workers):
            for index in range(len(instance.order_repeats)):
                names.append("theta " + str(instance.order_repeats[index][0]) + " " + str(instance.order_repeats[index][1]) + " " + str(j))
                obj_coeffs.append(-coef_ordenes_repetitivas)
                types.append("B")
                lower.append(0)
                upper.append(1)
                
    #add all variables to the model
    prob.variables.add(obj=obj_coeffs, lb=lower, ub=upper, types=types, names=names)



#### Defining model constraints

In [15]:

def add_constraints(prob, instance):
    n_workers = instance.n_workers
    n_orders = instance.n_orders

    #sum of Wxj must equal total hours worked by worker j
    for j in range(n_workers):
        indices = []
        values = []
        for x in [1, 2, 3, 4]:
            indices.append(f"W {x} {j}")
            values.append(1)
        for i in range(n_orders):
            indices.append(f"A {int(instance.orders[i].oid)} {j}")
            values.append(-1)
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['E'],
            rhs=[0],
            names=[f"total_hours_worker_{j}"]
        )

    #relation between Y_xj and W_xj
    for x in range(1, 4):
        for j in range(n_workers):
            indices = [f"Y {x} {j}", f"W {x} {j}"]
            values = [5, -1]
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[0],
                names=[f"lower_bound_ranges_1_to_3"]
            )
    for j in range(n_workers):
        indices = [f"Y 4 {j}", f"W 4 {j}"]
        values = [1, -1]
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[0],
            names=[f"lower_bound_range_4"]
        )
    for j in range(n_workers):
        indices = [f"W 1 {j}"]
        values = [1]
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[5],
            names=[f"upper_bound_range_1"]
        )
    for x in [2, 3]:
        for j in range(n_workers):
            indices = [f"W {x} {j}", f"Y {x - 1} {j}"]
            values = [1, -5]
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[0],
                names=[f"upper_bound_ranges_2_3"]
            )
    for j in range(n_workers):
        indices = [f"W 4 {j}", f"Y 3 {j}"]
        values = [1, -M]
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[0],
            names=[f"upper_bound_range_4"]
        )

    #no worker can do more than one task per shift
    for k in range(1, 31):
        for j in range(n_workers):
            indices = [f"X {k} {j}"]
            values = [1]
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[1],
                names=[f"one_task_per_shift_{k}_{j}"]
            )

    #no worker can work all 6 days
    for j in range(n_workers):
        indices = []
        values = []
        for d in range(1, 7):
            indices.append(f"V {d} {j}")
            values.append(1)
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[5],
            names=[f"max_5_days_worker_{j}"]
        )

    #link between V_dj and X_kj
    #lower bound
    for j in range(n_workers):
        for d in range(1, 7):
            indices = [f"V {d} {j}"]
            values = [1]
            for k in range(5 * d - 4, 5 * d + 1):
                indices.append(f"X {k} {j}")
                values.append(-1)
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[0],
                names=[f"v_lower_bound_link_x_{k}_{j}_day_{d}"]
            )

    #upper bound
    for j in range(n_workers):
        for d in range(1, 7):
            indices = [f"V {d} {j}"]
            values = [M]
            for k in range(5 * d - 4, 5 * d + 1):
                indices.append(f"X {k} {j}")
                values.append(-1)
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['G'],
                rhs=[0],
                names=[f"v_upper_bound_link_x_{k}_{j}_day_{d}"]
            )

    #fairness constraints: difference in hours <= 8

    #max work constraint
    for j in range(n_workers):
        indices = []
        values = []
        for i in range(n_orders):
            indices.append(f"A {int(instance.orders[i].oid)} {j}")
            values.append(1)
        indices.append("more_work")
        values.append(-1)
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[0],
            names=[f"fix_max_work_{j}"]
        )

    #min work constraint
    for j in range(n_workers):
        indices = []
        values = []
        for i in range(n_orders):
            indices.append(f"A {int(instance.orders[i].oid)} {j}")
            values.append(-1)
        indices.append("less_work")
        values.append(1)
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[0],
            names=[f"fix_min_work_{j}"]
        )

    #max - min <= 8
    indices = ["more_work", "less_work"]
    values = [1, -1]
    prob.linear_constraints.add(
        lin_expr=[[indices, values]],
        senses=['L'],
        rhs=[8],
        names=["max_minus_min_leq_8"]
    )

    #each order is assigned to at most one shift
    for i in range(n_orders):
        indices = []
        values = []
        for k in range(1, 31):
            indices.append(f"O {int(instance.orders[i].oid)} {k}")
            values.append(1)
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[1],
            names=[f"order_one_shift_{instance.orders[i].oid}_{k}"]
        )
    
    #there are pairs of conflicting orders (i, i') that no worker can perform consecutively:

    # delta_ikj <= O_ik:
    for i, iprime in instance.order_conflicts:
        for k in range(1, 31):
            for j in range(n_workers):
                indices = [f"delta {int(instance.orders[i].oid)} {k} {j}", f"O {int(instance.orders[i].oid)} {k}"]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"delta_triggered_if_O_{int(instance.orders[i].oid)}_{k}"]
                )

    # delta_ikj <= A_ij:
    for i, iprime in instance.order_conflicts:
        for k in range(1, 31):
            for j in range(n_workers):
                indices = [f"delta {int(instance.orders[i].oid)} {k} {j}", f"A {int(instance.orders[i].oid)} {j}"]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"delta_triggered_if_A_{int(instance.orders[i].oid)}_{j}"]
                )

    # delta_ikj <= X_kj:
    for i, iprime in instance.order_conflicts:
        for k in range(1, 31):
            for j in range(n_workers):
                indices = [f"delta {int(instance.orders[i].oid)} {k} {j}", f"X {k} {j}"]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"delta_triggered_if_X_{k}_{j}"]
                )

    # delta_ikj >= O_ik + A_ij + X_kj - 2:
    for i, iprime in instance.order_conflicts:
        for k in range(1, 31):
            for j in range(n_workers):
                indices = [
                    f"delta {int(instance.orders[i].oid)} {k} {j}",
                    f"O {int(instance.orders[i].oid)} {k}",
                    f"A {int(instance.orders[i].oid)} {j}",
                    f"X {k} {j}"
                ]
                values = [1, -1, -1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['G'],
                    rhs=[-2],
                    names=[f"delta_triggered_if_all_active_{k}_{j}"]
                )

    # delta_ikj + O_i'k+1 + A_i'j + X_k+1,j <= 3:
    for i, iprime in instance.order_conflicts:
        for k in range(1, 30):
            if k % 5 != 0:
                for j in range(n_workers):
                    indices = [
                        f"delta {int(instance.orders[i].oid)} {k} {j}",
                        f"O {int(instance.orders[iprime].oid)} {k+1}",
                        f"A {int(instance.orders[iprime].oid)} {j}",
                        f"X {k+1} {j}"
                    ]
                    values = [1, 1, 1, 1]
                    prob.linear_constraints.add(
                        lin_expr=[[indices, values]],
                        senses=['L'],
                        rhs=[3],
                        names=[f"no_conflict_orders_overlap_{int(instance.orders[i].oid)}_{int(instance.orders[iprime].oid)}_{j}"]
                    )

    # same as above but reversed order
    for i, iprime in instance.order_conflicts:
        for k in range(1, 30):
            if k % 5 != 0:
                for j in range(n_workers):
                    indices = [
                        f"delta {int(instance.orders[iprime].oid)} {k} {j}",
                        f"O {int(instance.orders[i].oid)} {k+1}",
                        f"A {int(instance.orders[i].oid)} {j}",
                        f"X {k+1} {j}"
                    ]
                    values = [1, 1, 1, 1]
                    prob.linear_constraints.add(
                        lin_expr=[[indices, values]],
                        senses=['L'],
                        rhs=[3],
                        names=[f"no_conflict_orders_overlap_{int(instance.orders[i].oid)}_{int(instance.orders[iprime].oid)}_{j}"]
                    )

    # if task i is correlated with i', then i' must be done right after i, on the same day
    # O_i'k+1 <= O_ik
    for i, iprime in instance.order_chains:
        for k in range(1, 30):
            indices = [
                f"O {int(instance.orders[iprime].oid)} {k+1}",
                f"O {int(instance.orders[i].oid)} {k}"
            ]
            values = [1, -1]
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[0],
                names=[f"respect_order_chain_{int(instance.orders[i].oid)}_{int(instance.orders[iprime].oid)}"]
            )

    # also: correlated order i' must not be first of the day
    for i, iprime in instance.order_chains:
        for k in [1, 6, 11, 16, 21, 26]:
            indices = [f"O {int(instance.orders[iprime].oid)} {k}"]
            values = [1]
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['E'],
                rhs=[0],
                names=[f"correlated_not_first_shift_{int(instance.orders[iprime].oid)}"]
            )

    # each day: max 5 shifts per worker
    for d in [1, 6, 11, 16, 21, 26]:
        for j in range(n_workers):
            indices = [f"X {k} {j}" for k in range(d, d+5)]
            values = [1] * 5
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[5],
                names=[f"max_shifts_per_day_{k}_{j}"]
            )


    #linked to variables A_ij, X_kj and O_ik:
    #delta_ikj = 1 iff X_kj = O_ik = A_ij = 1, modeled as:
    # 3 * delta_ikj <= X_kj + O_ik + A_ij
    # X_kj + O_ik + A_ij <= delta_ikj + 2

    #lower bound: 3 * delta_ikj <= X_kj + O_ik + A_ij
    for i in range(n_orders):
        for k in range(1, 31):
            for j in range(n_workers): 
                indices = [
                    f"delta {int(instance.orders[i].oid)} {k} {j}",
                    f"X {k} {j}",
                    f"O {int(instance.orders[i].oid)} {k}",
                    f"A {int(instance.orders[i].oid)} {j}"
                ]
                values = [3, -1, -1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"delta_lower_bound_link_A_{int(instance.orders[i].oid)}_{j}_X_{k}_{j}_O_{int(instance.orders[i].oid)}_{k}"]
                )

    #upper bound: X_kj + O_ik + A_ij <= delta_ikj + 2
    for i in range(n_orders):
        for k in range(1, 31):
            for j in range(n_workers): 
                indices = [
                    f"delta {int(instance.orders[i].oid)} {k} {j}",
                    f"X {k} {j}",
                    f"O {int(instance.orders[i].oid)} {k}",
                    f"A {int(instance.orders[i].oid)} {j}"
                ]
                values = [-1, 1, 1, 1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[2],
                    names=[f"delta_upper_bound_link_A_{int(instance.orders[i].oid)}_{j}_X_{k}_{j}_O_{int(instance.orders[i].oid)}_{k}"]
                )

    #relation H_i = sum_k O_ik
    for i in range(n_orders):
        indices = [f"H {int(instance.orders[i].oid)}"] + [f"O {int(instance.orders[i].oid)} {k}" for k in range(1, 31)]
        values = [1] + [-1] * 30
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['E'],
            rhs=[0],
            names=[f"link_H_{int(instance.orders[i].oid)}"]
        )

    #relation O_ik, A_ij and X_kj
    for i in range(n_orders):
        for k in range(1, 31):
            for j in range(n_workers):
                indices = [f"O {int(instance.orders[i].oid)} {k}", f"A {int(instance.orders[i].oid)} {j}", f"X {k} {j}"]
                values = [1, 1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[1],
                    names=[f"link_O_A_X_{int(instance.orders[i].oid)}_{k}_{j}"]
                )

    for k in range(1, 31):
        for j in range(n_workers):
            indices = [f"X {k} {j}"] + [f"delta {int(instance.orders[i].oid)} {k} {j}" for i in range(n_orders)]
            values = [1] + [-1] * n_orders
            prob.linear_constraints.add(
                lin_expr=[[indices, values]],
                senses=['L'],
                rhs=[0],
                names=[f"link_X_with_delta_{k}_{j}"]
            )

    #if an order is performed, it must be assigned exactly the correct number of workers
    for i in range(n_orders):
        indices = [f"H {int(instance.orders[i].oid)}"] + [f"A {int(instance.orders[i].oid)} {j}" for j in range(n_workers)]
        values = [int(instance.orders[i].n_workers)] + [-1] * n_workers
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['E'],
            rhs=[0],
            names=[f"assign_exact_workers_H_{int(instance.orders[i].oid)}"]
        )

    #each order must be done in at most one shift
    for i in range(n_orders):
        indices = [f"O {int(instance.orders[i].oid)} {k}" for k in range(1, 31)]
        values = [1] * 30
        prob.linear_constraints.add(
            lin_expr=[[indices, values]],
            senses=['L'],
            rhs=[1],
            names=[f"order_one_shift_{i}"]
        )

    #soft constraints if enabled
    if add_soft_variables:

        #worker conflicts: lambda_ijj'
        for i in range(n_orders):
            for j, jprime in instance.worker_conflicts:
                indices = [
                    f"lambda {int(instance.orders[i].oid)} {j} {jprime}",
                    f"A {int(instance.orders[i].oid)} {jprime}"
                ]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"lambda_{int(instance.orders[i].oid)}_{j}_{jprime}_if_A_{jprime}"]
                )
        for i in range(n_orders):
            for j, jprime in instance.worker_conflicts:
                indices = [
                    f"lambda {int(instance.orders[i].oid)} {j} {jprime}",
                    f"A {int(instance.orders[i].oid)} {j}"
                ]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"lambda_{int(instance.orders[i].oid)}_{j}_{jprime}_if_A_{j}"]
                )
        for i in range(n_orders):
            for j, jprime in instance.worker_conflicts:
                indices = [
                    f"lambda {int(instance.orders[i].oid)} {j} {jprime}",
                    f"A {int(instance.orders[i].oid)} {jprime}",
                    f"A {int(instance.orders[i].oid)} {j}"
                ]
                values = [1, -1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['G'],
                    rhs=[-1],
                    names=[f"lambda_{int(instance.orders[i].oid)}_{j}_{jprime}_if_both_A"]
                )

        #repetitive orders: theta_iiprimej
        for j in range(n_workers):
            for index in range(len(instance.order_repeats)):
                i, iprime = instance.order_repeats[index]
                indices = [f"theta {i} {iprime} {j}", f"A {i} {j}"]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"theta_{i}_{iprime}_{j}_if_A_{i}_{j}"]
                )
        for j in range(n_workers):
            for index in range(len(instance.order_repeats)):
                i, iprime = instance.order_repeats[index]
                indices = [f"theta {i} {iprime} {j}", f"A {iprime} {j}"]
                values = [1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['L'],
                    rhs=[0],
                    names=[f"theta_{i}_{iprime}_{j}_if_A_{iprime}_{j}"]
                )
        for j in range(n_workers):
            for index in range(len(instance.order_repeats)):
                i, iprime = instance.order_repeats[index]
                indices = [f"theta {i} {iprime} {j}", f"A {i} {j}", f"A {iprime} {j}"]
                values = [1, -1, -1]
                prob.linear_constraints.add(
                    lin_expr=[[indices, values]],
                    senses=['G'],
                    rhs=[-1],
                    names=[f"theta_{i}_{iprime}_{j}_if_both_A"]
                )


#### Building and solving the model

In [16]:

def build_lp(prob, instance):

    #add variables
    add_variables(prob, instance)
   
    #add constraints
    add_constraints(prob, instance)

    #set objective sense
    prob.objective.set_sense(prob.objective.sense.maximize)

    #write lp to file
    prob.write('crew_assignment.lp')


def solve_lp(prob):

    #define solver parameters
    #prob.parameters....

    #solve the lp
    prob.solve()


def show_solution(prob, instance):
    #get solution info through 'solution'

    #get solver status
    status = prob.solution.get_status_string(status_code=prob.solution.get_status())

    #get objective value
    obj_value = prob.solution.get_objective_value()

    print('objective function: ', obj_value, '(' + str(status) + ')')


def main():

    #read input data from file
    instance = load_instance()

    #define Cplex problem
    prob = cplex.Cplex()

    #build model
    build_lp(prob, instance)

    #solve model
    solve_lp(prob)

    print(instance.orders[0].n_workers)

    #get solution
    show_solution(prob, instance)

if __name__ == "__main__":
    main()


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.29 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3208 rows and 1222 columns.
MIP Presolve modified 2950 coefficients.
Reduced MIP has 15948 rows, 6040 columns, and 63287 nonzeros.
Reduced MIP has 5998 binaries, 42 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (83.70 ticks)
Probing time = 0.04 sec. (5.77 ticks)
Tried aggregator 2 times.
Detecting symmetries...
Aggregator did 1 substitutions.
Reduced MIP has 15947 rows, 6039 columns, and 63285 nonzeros.
Reduced MIP has 5998 binaries, 41 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (90.64 ticks)
Probing time = 0.03 sec. (5.39 ticks)
Clique table members: 37463.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.29 sec. (59